In [1]:
# 각 PSG 마다 total sleep time이 정리되어 있는 excel 파일이 있는 경우 실행 가능한 프로그램입니다.

SpO2_folder_path = 'C:/Users/smcsl/Desktop/PSG_study/Brain_age/Hypoxic_burden/Raw_data/SpO2/'
Events_folder_path = 'C:/Users/smcsl/Desktop/PSG_study/Brain_age/Hypoxic_burden/Raw_data/Events/'
Patients_excel_path = 'C:/Users/smcsl/Desktop/PSG_study/Brain_age/Hypoxic_burden/Raw_data/Patients_excel/'
Patients_excel_file_name = 'patients.xlsx'
tst_column_name = 'tst'
PSG_no_column_name = 'PSG_no'
Result_path = 'C:/Users/smcsl/Desktop/PSG_study/Brain_age/Hypoxic_burden/'
Result_name = 'Hypoxic_burden_excel'

if '\\' in SpO2_folder_path:
    raise SystemError('SpO2_folder_path 에 \ 있습니다. / 으로 바꿔주세요')
if '/' != SpO2_folder_path[-1]:
    raise SystemError('SpO2_folder_path 마지막에 / 붙여주세요')
if '\\' in Events_folder_path:
    raise SystemError('Events_folder_path 에 \ 있습니다. / 으로 바꿔주세요')
if '/' != Events_folder_path[-1]:
    raise SystemError('Events_folder_path 마지막에 / 붙여주세요')
if '\\' in Patients_excel_path:
    raise SystemError('Patients_excel_path 에 \ 있습니다. / 으로 바꿔주세요')
if '/' != Patients_excel_path[-1]:
    raise SystemError('Patients_excel_path 마지막에 / 붙여주세요')
if '\\' in Result_path:
    raise SystemError('Result_path 에 \ 있습니다. / 으로 바꿔주세요')
if '/' != Result_path[-1]:
    raise SystemError('Result_path 마지막에 / 붙여주세요')
if '.xlsx' != Patients_excel_file_name[-5:]:
    raise SystemError('Patients_excel_file_name 마지막에 .xlsx 붙여주세요')
    
import pandas as pd
import os, sys
dir_path = os.path.dirname(os.path.realpath('tools'))
sys.path.append(os.path.abspath(os.path.join(dir_path, os.pardir)))
import re
from tqdm import tqdm
import warnings
import hypoxic_burden
warnings.filterwarnings("ignore")

def find_file_by_PSG_no(folder, PSG_no):
    txt_re = re.compile(PSG_no.lower())
    try:
        return [file for file in folder if len(txt_re.findall(file.lower())) != 0][0]
    except:
        return False

SpO2_folder = os.listdir(SpO2_folder_path)
Events_folder = os.listdir(Events_folder_path)
try:
    patients = pd.read_excel(Patients_excel_path + Patients_excel_file_name).set_index([PSG_no_column_name])
except:
    raise SystemError('PSG_no_column_name 정확한지 확인하십시오')

for file in SpO2_folder:
    if ',' not in file:
        raise SystemError(file, ' 파일 이름에 PSG number 근처로 쉼표(,) 확인 부탁드립니다. 파일 이름 형식은 메뉴얼 참고 부탁드립니다.')
for file in Events_folder:
    if '-' not in file:
        raise SystemError(file, ' 파일 이름에 PSG number 근처로 띄어쓰기 또는 짝대기(-) 확인 부탁드립니다. 파일 이름 형식은 메뉴얼 참고 부탁드립니다.')

SpO2_PSG_no_list = [file.split(',')[0].upper() for file in SpO2_folder]
Events_PSG_no_list = [file.split(' ')[-1].split('-')[0].upper() for file in Events_folder]
Excel_PSG_no_list = list(patients.index.map(lambda x: x.upper()))
PSG_no_list = list(set(SpO2_PSG_no_list) & set(Events_PSG_no_list) & set(Excel_PSG_no_list))

hypoxic_burden_df = pd.DataFrame(columns = ['Total sleep time (h)', 'Apnea-specific hypoxic burden (%min/h)', 'Hypopnea-specific hypoxic burden (%min/h)', \
    'Total hypoxic burden (%min/h)'], index = PSG_no_list)

miss_SpO2 = list(set(SpO2_PSG_no_list) - set(PSG_no_list))
miss_Events = list(set(Events_PSG_no_list) - set(PSG_no_list))
miss_Excel = list(set(Excel_PSG_no_list) - set(PSG_no_list))

if len(miss_SpO2):
    print('다음이 포함된 SpO2 파일들은 이름 형식이 잘못되어 hypoxic burden 계산이 되지 않습니다.')
    print(miss_SpO2)
    print('수정하기 위해서는 상단의 정지 표시의 "Interrupt"를 눌러주시고 파일 이름 형식 수정 후 다시 실행하십시오')
    print('-------------------------------------------------------------------------------------------------------------------------------------------')
if len(miss_Events):
    print('다음이 포함된 Event 파일들은 이름 형식이 잘못되어 hypoxic burden 계산이 되지 않습니다.')
    print(miss_Events)
    print('수정하기 위해서는 상단의 정지 표시의 "Interrupt"를 눌러주시고 파일 이름 형식 수정 후 다시 실행하십시오')
    print('-------------------------------------------------------------------------------------------------------------------------------------------')
if len(miss_Excel):
    print('다음이 포함된 SpO2, Event 파일들을 찾을 수가 없어, hypoxic burden 계산이 되지 않습니다.')
    print(miss_Excel)
    print('수정하기 위해서는 상단의 정지 표시의 "Interrupt"를 눌러주시고 파일 이름 형식 수정 후 다시 실행하십시오')
    print('-------------------------------------------------------------------------------------------------------------------------------------------')

print('환자 이름이 SpO2 파일과 excel에서 다른 PSG_no 들입니다. 확인하시고 수정하기 위해서 상단의 정지 표시의 "Interrupt"를 눌러주시고 수정 후 다시 실행하십시오.')
for file in SpO2_folder:
    psg_no = file.split(',')[0].upper()
    if psg_no in PSG_no_list:
        if file[file.index(' ')+1:].split('_')[0] != patients.loc[psg_no, '이름']:
            print(psg_no + '의 SpO2 파일에서의 이름은 ' + file[file.index(' ')+1:].split('_')[0] + ' 이고, excel 파일에서의 이름은 ' + patients.loc[psg_no, '이름'] + ' 입니다.')
print('-------------------------------------------------------------------------------------------------------------------------------------------')

print('환자 이름이 Event 파일과 excel에서 다른 PSG_no 들입니다. 확인하시고 수정하기 위해서 상단의 정지 표시의 "Interrupt"를 눌러주시고 수정 후 다시 실행하십시오.')
for file in Events_folder:
    psg_no = file.split(' ')[-1].split('-')[0].upper()
    if psg_no in PSG_no_list:
        if file[:file.rindex(' ')] != patients.loc[psg_no, '이름']:
            print(psg_no + '의 Event 파일에서의 이름은 ' + file[:file.rindex(' ')] + ' 이고, excel 파일에서의 이름은 ' + patients.loc[psg_no, '이름'] + ' 입니다.')
print('-------------------------------------------------------------------------------------------------------------------------------------------')
            
for PSG_no in tqdm(PSG_no_list):
    Event_file = find_file_by_PSG_no(Events_folder, PSG_no)
    SpO2_file = find_file_by_PSG_no(SpO2_folder, PSG_no)
    if Event_file and SpO2_file:
        SpO2_path = SpO2_folder_path + SpO2_file
        Resp_event_path = Events_folder_path + Event_file
        try:
            tst = patients.loc[PSG_no, tst_column_name]
        except:
            raise SystemError('tst_column_name 정확한지 확인하십시오')

        psg = hypoxic_burden.PSG(PSG_no, tst, SpO2_path, Resp_event_path)

        hypoxic_burden_df.loc[PSG_no, 'Total sleep time (h)'] = tst
        hypoxic_burden_df.loc[PSG_no, ['Apnea-specific hypoxic burden (%min/h)', 'Hypopnea-specific hypoxic burden (%min/h)', \
            'Total hypoxic burden (%min/h)']] = psg.hypoxic_burden()
    else:
        raise SystemError('Missing Report or SpO2 or Event file of ' + PSG_no)

hypoxic_burden_df.index.name = 'PSG_no'
hypoxic_burden_df.to_excel(Result_path + Result_name + '.xlsx')

ModuleNotFoundError: No module named 'pandas'

: 